In [1]:
# !pip install multiprocess # 이거 절대로 쓰면 안된다
                            # 써봤다가 노트북(쥬피터 노트북이 아니다) 나갈 뻔했다

In [ ]:
import multiprocessing as mp
import numpy as np
import os

'''
    multiprocessing 모듈은 윈도우와 유닉스/리눅스에서 다르게 작동한다
    <윈도우>
    윈도우의 경우 fork() system call이 없어 새로운 프로세스를 시작(start)할 경우, 새로운 파이썬 인터프리터를 시작함과 동시에 전체 프로그램을 처음부터 다시 실행한다
    따라서 'if __name__ == "__main__":'을 명시하지 않으면 프로세스 생성문을 계속 실행되기 때문에 프로세스의 생성이 무한히 반복된다
    처음 파이썬 프로그램을 시작하여 생성된 첫번째 프로세스는 __name__에 "__main__"이 들어있지만 두번째 프로세스부터는 "__mp_main__"이 들어있다
    따라서 'if __name__ == "__main__":'을 명시하여 코드를 작성하면, 새로이 시작된 프로세스가 프로그램의 처음부터 다시 실행하더라도 if문 안쪽의 (여기서는)mp.Pool(8)을 실행하지 않아 프로세스가 무한히 생성되는 것을 막는다
    
    <유닉스/리눅스>
    유닉스/리눅스의 경우 fork() system call이 존재하기 때문에 새로운 프로세스 생성 명령이 실행되면 현재 프로세스를 복사하여 새로운 프로세스를 생성하고, 이 때 원본이 되는 프로세스를 "부모 프로세스" 복사된 프로세스를 "자식 프로세스"라고 한다
    생성된 자식 프로세스는 프로그램의 처음부터 다시 실행하는 것이 아닌 mp.Pool(8)와 같은 프로세스 생성 명령 바로 다음 명령부터 실행하기 때문에 프로세스가 무한정 생성되지 않는다

    <쥬피터에서 멀티-프로세싱이 안되는 이유>
    "윈도우"에서 파이썬 파일(.py)을 사용한 일반적인 멀티-프로세싱 실행 과정은 아래와 같다
    1. 새로운 프로세스 생성 명령을 호출하면 파이썬은 해당 명령을 호출한 (실제로 그런 것은 아니지만 이하 '부모 프로세스'라고 칭함)main module(.py 파일 혹은 전체 파이썬 코드가 저장된 파일)과 배정된 작업을 처리하기 위한 (이하 '데이터'라고 칭함)인자(데이터)를 pickle(serialize, 직렬화)하여 새로운 프로세스로 보낸다
    2. 새로운 프로세스(이하 '자식 프로세스'라 칭함)는 새로운 파이썬 인터프리터를 실행하고 넘겨받은 main module(.py 파일)을 다시 import 한다(즉, .py 파일에 적힌 코드를 처음부터 다시 실행한다는 것이다)
    3. 이후, 자식 프로세스는 전달받은 데이터를 unpickle(deserialize, 비직렬화)하고, 이를 입력 데이터로 하여 배정된 작업 함수를 호출한다
    4. 자식 프로세스가 main modeule(.py 파일)의 실행을 완료하면, 배정된 작업(mp_func())에 해당하는 함수의 반환된 결과를 pickle(serialize, 직렬화)하여 부모 프로세스에게 보낸다
    5. 부모 프로세스는 전달 받은 결과물을 unpickle(deserialize, 비직렬화)하여 취합하고, 나머지 파이썬 코드를 계속 실행한다
    
    헌데 쥬피터 노트북은(랩도 마찬가지) 일반적인 파이썬 스크립트가 아닌, 백그라운드에서 상호작용 가능한 파이썬 커널을 실행하고 있는 웹 애플리케이션이다
    이는 쥬피터 노트북에서의 main module(여기서는 .ipynb 파일)이 일반적인 파이썬 스크립트가 아니라 파이썬 객체임을 의미한다
    이러한 파이썬 객체는 pickle(serialize, 직렬화)할 수 없고 따라서 이를 새로운 프로세스에 보내줄 수 없다
    따라서 새로운 프로세스가 생성될 수 없으며, 이것이 쥬피터 상에서 (if문을 둘째치고서라도)멀티-프로세싱이 작동하지 않는 이유다

    <해결방법(4가지가 있지만 그 중 가장 간편한 방법)>
    멀티 프로세싱으로 처리할 작업에 해당하는 함수를 따로 파이썬 파일(.py)로 생성하고 그것을 불러온다
    위 방법이 가능한 이유는 아래와 같다
    1. 윈도우 상에서 새로운 프로세스를 생성할 때 main module을 넘겨준다는 했는데, 이는 새로운 프로세스에 배정한 작업 함수가 main module내에 작성되어있기 때문이다
    --> 따라서 작업에 해당하는 함수를 .py 파일로 별도로 생성하게 되면 pickle이 가능해져 main module을 새로운 프로세스에 전달해 줄 수 있게 된다
    2. 작업을 별도의 .py 파일로 생성할 때 "작업에 해당하는 함수만" 작성하였고 "프로세스 생성문은 작성하지 않았기 때문에" 무한 프로세스 생성 루프에 빠지지 않는다(빠질 수 없다)
    
    <참고>
    https://bobswinkels.com/posts/multiprocessing-python-windows-jupyter/
    how to use pool.map in jupyter notebook : 검색 문구
'''

def square(x):
    return np.square(x)

# 아래의 if문을 사용하지 않으면 무한 오류에 빠지게 된다
# 쥬피터에서는 아래의 if문을 사용해도 멀티 프로세싱이 수행되지 않는다
if __name__ == "__main__":
    x = np.arange(64) # 0 ~ 63까지의 정수 생성
    print(x, type(x), x.dtype)
    print("Current Host System cpu core number :", mp.cpu_count()) # 현재 호스트 시스템의 cpu(코어) 개수를 반환한다
                                                                   # 그러나 이것이 현재 프로세스가 사용 가능한 cpu(코어)의 개수를 의미하지는 않는다
    pool = mp.Pool(8) # 8개의 프로세스를 가지는 프로세스 풀을 생성한다
    squared = pool.map(square, [x[8*i:8*i+8] for i in range(8)]) # 총 64개의 숫자를 8개씩 분할하여 각 프로세스에 배정한다
    print(squared)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] <class 'numpy.ndarray'> int32
Current Host System cpu core number : 16


In [8]:
import numpy as np
import multiprocessing as mp
import os
from lib.worker import square

x = np.arange(64) # 0 ~ 63까지의 정수 생성
print(x, type(x), x.dtype)
print("Current Host System cpu core number :", mp.cpu_count()) # 현재 호스트 시스템의 cpu(코어) 개수를 반환한다
                                                               # 그러나 이것이 현재 프로세스가 사용 가능한 cpu(코어)의 개수를 의미하지는 않는다
# with 구문을 사용하지 않아도 코드 실행에 아무런 문제는 없지만,
# with을 사용하는 것이 자원 관리 측면에서 훨씬 더 좋다
with mp.Pool(8) as pool:
    squared = pool.map(square, [x[8*i:8*i+8] for i in range(8)]) # 총 64개의 숫자를 8개씩 분할하여 각 프로세스에 배정한다
                                                                 # 각 프로세스가 수행해야 하는 작업(함수)과 작업물(데이터)을 명시한다
for i in range(len(squared)):
    print(squared[i])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] <class 'numpy.ndarray'> int32
Current Host System cpu core number : 16
[ 0  1  4  9 16 25 36 49]
[ 64  81 100 121 144 169 196 225]
[256 289 324 361 400 441 484 529]
[576 625 676 729 784 841 900 961]
[1024 1089 1156 1225 1296 1369 1444 1521]
[1600 1681 1764 1849 1936 2025 2116 2209]
[2304 2401 2500 2601 2704 2809 2916 3025]
[3136 3249 3364 3481 3600 3721 3844 3969]


In [16]:
import numpy as np
import multiprocessing as mp
from lib.worker import square2

processes = [] # 프로세스 풀 생성
queue = mp.Queue() # 프로세스가 반환하는 결과물을 취합하기 위한 큐 생성
x = np.arange(64)

print("Current available Host System cpu core number :", mp.cpu_count())

# mp.Pool()은 어떤 프로세스에게 어떤 작업을 배정할지를 자동으로 결정하는 반면,
# mp.Process()는 어떤 프로세스에게 어떤 작업을 배정할지를 수동으로 지정한다
# 프로세스 작업 수행 과정은 CMD에서 확인할 수 있다(.run() 메서드를 사용하면 직접 출력해 볼 수 있다)
for i in range(8):
    start_index = 8 * i
    # multiprocessing에서 process는 Process 객체가 생성되고 이 객체의 start() 메서드가 호출되었을 때 소환된다(spawn)
    # Process 객체는 분리된 프로세스 내에서 실행중인 활동을 나타낸다
    proc = mp.Process(target=square2, args=(i, x[start_index:start_index+8], queue)) # 프로세스 번호, 작업물, 결과물 저장 큐를 넘겨준다
    # proc.run() # 프로세스의 활동을 출력한다
                 # 이 메서드를 사용하지 않으면 (쥬피터의 경우) CMD에 프로세스 활동이 출력된다
    proc.start() # 프로세스를 시작한다(프로세스가 실제로 생성된다)
    processes.append(proc) # 추후 프로세스들을 제어하기 위해 프로세스들을 추적한다

for proc in processes:
    proc.join() # 프로세스가 종료될 때까지 기다린다

for proc in processes:
    proc.terminate() # 프로세스를 종료한다(자원 반환은 수행하지 않는다)
                     # 부모 프로세스를 종료하더라도 그 밑의 자식 프로세스들은 종료되지 않는다(이 경우 자식 프로세스들은 "기아"가 된다)

for proc in processes:
    proc.close() # 프로세스 객체를 해체하여 그것과 관련된 모든 자원들을 반환한다

results = []
while not queue.empty(): # 큐가 비어있지 않을 경우, 큐 안의 결과물을 반환한다
    results.append(queue.get())

for i in range(len(results)):
    print(results[i])

Current available Host System cpu core number : 16
[256 289 324 361 400 441 484 529]
[ 0  1  4  9 16 25 36 49]
[1024 1089 1156 1225 1296 1369 1444 1521]
[ 64  81 100 121 144 169 196 225]
[1600 1681 1764 1849 1936 2025 2116 2209]
[576 625 676 729 784 841 900 961]
[2304 2401 2500 2601 2704 2809 2916 3025]
[3136 3249 3364 3481 3600 3721 3844 3969]


In [6]:
import torch
import torch.nn as nn

''' 
    < 실험 결과 >
    - layer2 계층에서 도출된 출력으로 손실을 계산하여 역전파를 수행했을 경우
    ==> layer2 계층에 대한 기울기만 계산된다
    - med_out 계층에서 도출된 출력으로 손실을 계산하여 역전파를 수행했을 경우
    ==> med_out과 layer1 계층에 대한 기울기만 계산된다
    - layer2와 med_out 계층에서 도출된 출력으로 각각 손실을 계산하여 합산한 후 역전파를 수행했을 경우
    ==> 모든 계층에 대한 기울기가 계산된다
'''
class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 10)
        self.med_out = nn.Linear(10, 4)
        self.layer2 = nn.Linear(10, 10)
        
    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.med_out(out1)
        out3 = self.layer2(out1.detach())
        return out2, out3
    
model = TestModel()

x = torch.randn(1, 10)
# a = model(x)[0]
# a = model(x)[1]
# a.mean().backward()
a, b = model(x)
total = a.mean() + b.mean()
total.backward()
print("layer1's gradient : ", model.layer1.weight.grad)
print("med_out's gradient : ",model.med_out.weight.grad)
print("layer2's gradient : ",model.layer2.weight.grad)

layer1's gradient :  tensor([[ 0.1195, -0.1200, -0.2111,  0.0630,  0.1049,  0.0318, -0.0020, -0.0011,
          0.0909,  0.1271],
        [ 0.0288, -0.0289, -0.0508,  0.0152,  0.0253,  0.0076, -0.0005, -0.0003,
          0.0219,  0.0306],
        [-0.1314,  0.1320,  0.2321, -0.0692, -0.1154, -0.0349,  0.0022,  0.0012,
         -0.0999, -0.1398],
        [ 0.1182, -0.1187, -0.2088,  0.0623,  0.1038,  0.0314, -0.0020, -0.0011,
          0.0899,  0.1258],
        [ 0.1106, -0.1111, -0.1953,  0.0583,  0.0971,  0.0294, -0.0019, -0.0010,
          0.0841,  0.1176],
        [ 0.0527, -0.0529, -0.0931,  0.0278,  0.0463,  0.0140, -0.0009, -0.0005,
          0.0401,  0.0560],
        [-0.0833,  0.0836,  0.1470, -0.0439, -0.0731, -0.0221,  0.0014,  0.0008,
         -0.0633, -0.0886],
        [ 0.0431, -0.0433, -0.0761,  0.0227,  0.0378,  0.0114, -0.0007, -0.0004,
          0.0328,  0.0458],
        [-0.0648,  0.0651,  0.1144, -0.0341, -0.0569, -0.0172,  0.0011,  0.0006,
         -0.0493, -0.0689]

In [ ]:
''' F.normalize() 작동 방식 실험 1 '''

b = torch.randn(2, 5)
l2_norm_b = b[0].pow(2).sum(dim=0).sqrt()
print("b:", b)
# print(b[0].pow(2))
# print(b[0].pow(2).sum(dim=0))
print("b[0]'s l2 norm1:", b[0].pow(2).sum(dim=0).sqrt()) # l2 norm
print("b[0]'s l2 norm2:",b[0].norm(p=2))
print("b's l2 norm for dim=0:", b.norm(p=2, dim=0)) # dim=0는 열벡터를 대상으로 l2-norm을 계산한다
                                                    # 정확히는 dim=0를 구성하는 원소들로 (열)벡터를 만들고, 그것들 각각에 대하여 l2-norm을 계산한다
print("b's l2 norm for dim=1:", b.norm(p=2, dim=1)) # dim=1은 행벡터를 대상으로 l2-norm을 계산한다
                                                    # 정확히는 dim=1를 구성하는 원소들로 (열)벡터를 만들고, 그것들 각각에 대하여 l2-norm을 계산한다
print("result:", b[0] / l2_norm_b)
print(F.normalize(b, dim=0)) # dim=0는 열벡터를 대상으로 l2-norm을 구한 후, 그것으로 해당 열벡터를 나누어준다
                             # 정확히는 dim=0를 구성하는 원소들로 열벡터를 만들어 그것들 각각에 대해 l2-norm을 계산한 후, 그 결과를 그것에 대응하는 열벡터에 나눠준다
print(F.normalize(b, dim=1)) # dim=1는 행벡터를 대상으로 l2-norm을 구한 후, 그것으로 해당 열벡터를 나누어준다
                             # 정확히는 dim=1를 구성하는 원소들로 열벡터를 만들어 그것들 각각에 대해 l2-norm을 계산한 후, 그 결과를 그것에 대응하는 열벡터에 나눠준다

In [ ]:
''' F.normalize() 작동 방식 실험 2 '''

(b[0][0])**2 + (b[1][0])**2, ((b[0][0])**2 + (b[1][0])**2).sqrt()

In [ ]:
''' F.normalize() 작동 방식 실험 3 '''

temp = ((b[0][0])**2 + (b[1][0])**2).sqrt()
b[:,0], b[:,0] / temp, F.normalize(b, dim=1), b[0] / b[0].norm(p=2)

In [4]:
import torch
from torch import nn, optim
import numpy as np
from torch.nn import functional as F
import torch.multiprocessing as mp
import gymnasium as gym

class ActorCritic(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(4, 25)
        self.l2 = nn.Linear(25, 50)
        self.actor_lin1 = nn.Linear(50, 2)
        self.l3 = nn.Linear(50, 25)
        self.critic_lin1 = nn.Linear(25, 1)

    def forward(self, x):
        x = F.normalize(x, dim=0) # F.normalize()에서 도출되는 결과의 범위는 [-1.0, 1.0]이다
        y = F.relu(self.l1(x))
        y = F.relu(self.l1(y))
        actor = F.log_softmax(self.actor_lin1(y), dim=0) # 음의 로그 확률 값을 모델 단에서 미리 계산한다(정확히는 그냥 로그 확률이다)
        c = F.relu(self.l3(y.detach())) # 여기서 계산 그래프가 분리된다
        critic = torch.tanh(self.critic_lin1(c)) # 가치 함수 값을 tanh을 사용하여 [-1.0, 1.0] 구간의 값으로 변환시켜준 것은
                                                 # 이익 계산에서의 Returns를 정규화하여 [-1.0, 1.0] 구간의 값으로 변환하기 때문이다
        return actor, critic

[]


In [ ]:
def worker(t, worker_model, counter, params):
    worker_env = gym.make("CartPole-v1") # 환경 불러오기
    worker_env.reset() # 환경 초기화
    worker_opt = optim.Adam(lr=1e-4, params=worker_model.parameters())

    for i in range(params["epochs"]):
        state_values, logprobs, rewards = run_episode(worker_env, worker_model)
        actor_loss, critic_loss, ep_len = update_params(worker_opt, state_values, logprobs, rewards)
        counter.value = counter.value + 1

def run_episode(worker_env, worker_model):
    state = torch.from_numpy(worker_env.state).float()
    state_values, logprobs, rewards = [], [], []
    done = False # 에피소드 종료 여부

    while (done == False):
        policy, state_value = worker_model(state)
        state_values.append(state_value)
        logits = policy.view(-1) # 1차원 텐서로 변환한다
        action_dist = torch.ditributions.Categorical(logtis=logits) # 카테고리컬 분포는 시행 횟수 n이 1인 다항분포와 동일한 분포이다
                                                                    # 여기서의 역할은 주어진 로짓을 확률분포로 변환하여 이 확률분포를 토대로 표본을 추출할 수 있도록 하는 것이다
        action = action_dist.sample()
        logprob_ = logits[action]
        logprobs.append(logprob_)
        next_state, _, done, _, _ = woker_env.step(action.numpy())
        state = torh.from_numpy(next_state).float()
        if done:
            reward = -10
            worker_env.reset()
        else:
            reward = 1.0
        rewards.append(reward)

    return state_values, logprobs, rewards

def update_params(worker_opt, state_values, logprobs, rewards, clc=0.1, gamma=0.95):
    rewards = torch.tensor(rewards).flip(dim=(0,)).view(-1)
    logprobs = torch.stack(logprobs).flip(dims=(0,)).view(-1) # torch.stack 대신 torch.tensor를 사용해도 된다
    state_values = torch.stack(values).flip(dims=(0,)).view(-1) # torch.stack 대신 torch.tensor를 사용해도 된다
    Returns = [] # 반환값 저장
    ret_ = torch.tensor([0])
    for r in range(rewards.shape[0]): # 보상의 개수만큼 반복을 수행한다
        ret_ = rewards[r] + gamma * ret_ # 에피소드의 마지막 타임 스텝부터 반환값을 계산한다
        Returns.append(ret_)

    Returns = torch.stack(Returns).view(-1) # 텐서가 원소인 리스트를 torch.tensor를 통해 텐서로 변환하면 오류가 발생하는데, torch.stack을 사용하면 오류없이 텐서로 변환할 수 있다
    Returns = F.normalize(Returns, dim=0) # 반환값들에 대해 정규화를 수행하여 [-1.0, 1.0] 구간의 값으로 변환한다
                                          # 이것때문에 비평자의 출력에 tanh를 적용한 것이다
    actor_loss = -1 * logprobs * (Returns - values.detach())
    critic_loss = torch.pow(values - Returns, 2)
    loss = actor_loss.sum() + clc * critic_loss.sum() # 행위자가 비평자보다 더 빨리 학습하도록 하기 위해 clc=0.1을 곱한다
                                                      # 비평자의 전체 손실 중 일부로만 역전파를 수행하여 비평자의 학습을 지연시킨다
    # 역전파 수행
    worker_opt.zero_grad()
    loss.backward()
    worker_opt.step()

    return actor_loss, critic_loss, len(rewards) 

In [ ]:
master_node = ActorCritic()
master_node.share_memory()
processes = []
params = {
    "epochs":1000,
    "n_workers":7
}

# 여기서의 mp.Value는 python의 multiprocessing 모듈에 존재하는 클래스이다
# torch.multiprocessing이 multiprocessing의 warraper이자 100% 호환되기 때문에 torch.multiprocessing에서도 이용 가능한 것이다
counter = mp.Value("i", 0) # 공유 메모리(shared memory)로부터 할당된 ctypes(C언어와 호환되는 자료형이다) 객체를 반환한다
                           # "i"는 typecode_or_type의 인자로, 이것은 반환되는 객체의 자료형을 결정한다
                           # typecode_or_type은 ctypes 형이나 또는 array 모듈에서 사용되는 종류의 단일 문자 typecode가 될 수 있다

https://docs.python.org/3/library/multiprocessing.html#multiprocessing.Value
https://devocean.sk.com/blog/techBoardDetail.do?ID=163669

In [11]:
import torch

a = [1,2,3,4,5,6]
b = []
for i in range(6):
    b.append(torch.tensor(a[i]))
# torch.stack(b)
c = torch.tensor(b)
print(c, c.shape)

tensor([1, 2, 3, 4, 5, 6]) torch.Size([6])
